# Reference 
* [paper in arXiv2016](https://arxiv.org/abs/1602.07360)
* Reference to [MLT Artificial Intelligence's YouTube](https://www.youtube.com/watch?v=DbPbYGopC2c)
* Referemce to [gsp-27's github](https://github.com/gsp-27/pytorch_Squeezenet/blob/master/model.py)
* Reference to [2D-SqueezeNet of pytorch vision](https://github.com/pytorch/vision/blob/main/torchvision/models/squeezenet.py)
* Reference to [VITALab's blog](https://vitalab.github.io/article/2018/03/15/squeezeNet.html)
* Reference to [arvention's github](https://github.com/arvention/SqueezeNet-PyTorch/blob/master/model.py)
* Reference to [gsp-27's github](https://github.com/gsp-27/pytorch_Squeezenet/blob/master/model.py)
* Reference to [JINSOL KIM's blog](https://gaussian37.github.io/dl-concept-squeezenet/)
* Reference to [PR-144 Youtube](https://youtu.be/WReWeADJ3Pw)
* Reference to [Matthijs Hollemans's post](https://machinethink.net/blog/mobile-architectures/)

In [ ]:
import numpy as np 

import torch 
import torch.nn as nn 
import torch.nn.init as init

In [ ]:
# (ref) https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 1. Contribution and Background

<b>[의의]</b>
* ImageNet에서 ```AlexNet 수준의 accuracy```를 달성하였지만 ```AlexNet 보다 50배 작은 파라미터 수```를 이용하였다는 것에 의미가 있음. 
* 또한, hyper-parameter를 어떻게 조합하느냐에 따라 ```성능과 파라미터의 개수 사이의 관계```를 확인한 것에 의의가 있음 

<b>[배경]</b>
* 시간이 흐를수록 딥러닝 모델의 ```accuracy가 좋아지는 반면```, 모델의 ```사이즈가 점점 더 커지는 문제``` 발생. 
* 이는 모바일 및 임베딩 환경 같이 저사양 컴퓨팅 환경에서 사용하기 어렵게 만듬. 
* 이러한 배경 속에서 ```quantization```이나 ```모델 경량화```에 많은 관심이 생기게 되었습니다.

# 2. Preliminaries
스퀴즈넷에서는 <b>micro architecture</b>와 <b>macro architecture</b> 측면으로 나누어서 전체 내용을 설명하고 있음. <br/>

<b>[micro architecture]</b>
* $1 \times 1 $ convolution 
* $3 \times 3 $ convolution 
* skip connection 

<b>[macro architecture]</b>
* block stacking

## 2-1. micro architecture 

### (1) $1\times1$ conv (=sperable/point-wise convolution)
* 1x1 convolution filter가 channel 방향의 multi-layer perceptron과 같은 역할을 한다는 점. 
    * (왜?) 1x1 convolution의 경우 <b>각각의 채널에</b> element-wise로 multiplication 한 후에 합하게 된다. 이 연산은 채널 방향의 multi-layer perceptron 즉, FC layer와 같다는 뜻
* $1\times1$ filter 하나가 채널 방향으로  ```dot product``` 함으로써 perceptron node 하나를 만드는 것과 같다.
    * 대신 파라미터 개수 = $1\times1\times\text{num_channel}$로써 파라미터 개수가 dense layer 보다 작다. 

<img src="https://miro.medium.com/max/1400/1*dNaikOfrGzUaJ2EzRIl4tw.png" width=480> <br/>
(source: [Raj Sakthi's medium](https://medium.com/analytics-vidhya/talented-mr-1x1-comprehensive-look-at-1x1-convolution-in-deep-learning-f6b355825578))


In [ ]:
input = torch.randn(1, 192, 64, 64)
_, n_c, h, w = input.size() 

pw_conv = nn.Conv2d(in_channels= n_c,
                    out_channels = 1, 
                    kernel_size=1, 
                    stride=1,
                    padding=0,
                    bias=False,
                    )

output = pw_conv(input)
print(output.size())
print(f"# of learnable parameters: {count_parameters(pw_conv)}")

torch.Size([1, 1, 64, 64])
# of learnable parameters: 192


### (2) $3\times3$ conv

<b>[[Filter factorization](https://medium.com/@dmangla3/designing-faster-neural-networks-e1f1dc026533)]</b><br/>
어떤 필터를 사용할 것인가? 
* 초기 딥러닝 모델에서는 넓은 receptive field를 위해 $5\times5$ 혹은 $7\times7$ filter를 사용했다. 하지만, 요즘은 $3\times3$ filter를 사용하는 것이 주류다. 
* 어찌 보면 잘못된 선택 처럼 보이지만, 보다 작은 필터(e.g., $3\times3$)를 레이어 방향으로 순차적으로 적용하면 $5\times5$ filter 를 적용할 때와 똑같은 효과의 receptive fields를 얻을 수 있다. 

<img src="https://miro.medium.com/max/1314/1*PsvllwB__wjKpHdXTr0oKw.jpeg" width=480> <br/>
(source:[Amit Singh Bhatti's medium](https://medium.datadriveninvestor.com/how-neural-nets-fell-in-love-with-computer-hardwares-79a4617c8f3a), Decomposing larger filters into smaller filters.)

* 위 그림의 경우 $5\times5$ filter를 사용하려면 25-weight 이 필요하다. 
* 반면, $3\times3$ filter 두 개로 분해하면 ($3\times3 + 3\times3$)=18-weight 로 파라미터 개수가 더 적다. 

<b>[Other benefits]</b><br/>
* 사이즈가 작은 만큼 더욱 지엽적으로(highly local) 특징을 뽑을 수 있다. 즉, 작고 세밀한(fine-grained) 특징을 잡기에 유리하다. 
* Kernel을 연쇄적으로 사용하면(using kernel sequentially = increasing number of layers) 더 복잡한 특징(feature)를 학습할 수 있다.
    * $3\times3$ filter를 사용하면 activation function을 2번 거친다.
    * $5\times5$ filter를 사용하면 activation function을 1번 거친다. 

<b>[Example]</b>
* 1개 layer의 5x5 필터는 2개 layer의 3x3 필터로 대체될 수 있음.

<img src="https://oi.readthedocs.io/en/latest/_images/conv_factorization.png" width=480><br/>
(source:[bskyvision's blog](https://bskyvision.com/504))


In [ ]:
input = torch.randn(1, 1, 7, 7)
_, n_c, h, w = input.size() 


conv_5x5 = nn.Conv2d(in_channels= n_c,
                    out_channels = 1, 
                    kernel_size=5, 
                    stride=1,
                    padding=0,
                    bias=False,
                    )
output_5x5 = conv_5x5(input)
print(output_5x5.size())
print(f"# of learnable parameters: {count_parameters(conv_5x5)}")


conv_3x3 = nn.Sequential(
    nn.Conv2d(n_c, 1, 3, 1, 0, bias=False),
    nn.Conv2d(1, 1, 3, 1, 0, bias=False),
)
output_3x3 = conv_3x3(input)
print(output_3x3.size())
print(f"# of learnable parameters: {count_parameters(conv_3x3)}")

torch.Size([1, 1, 3, 3])
# of learnable parameters: 25
torch.Size([1, 1, 3, 3])
# of learnable parameters: 18


### (3) skip connection 
```skip connection```의 관점은 ResNet과 Segmentation에서 볼 수 있음. </br> 

<b>[Segmentation에서의 관점]</b>
* Segmentation에서는 resolution 정보가 굉장히 중요함. 하지만, deep layer로 계층이 내려갈수록 downsampling 과정에서 본래의 spatial resolution 정보가 소실됨.  
* 이를 다시 회복하기 위한 upsampling 과정에서 long skip connection을 통해 인코딩 전의 같은 레이어 레벨의 resolution 정보를 더해 줌으로써 완전한 spatial resolution을 회복하는 데 도움을 줌.

<img src="https://theaisummer.com/static/2c373d3667071700748bf451c4e62b78/7f018/long-skip-connection.jpg" width=480> </br>
(source: [AI SUMMER](https://theaisummer.com/skip-connections/))


</br>


<b>[ResNet에서의 관점]</b>
* 위와 비슷한 컨셉으로 작동한다. 
* Vanishing gradient 문제를 skip connection을 통해 개선한 것에 의미가 있고, 
* 혹자는 Optimization 관점에서 loss function을 smoothing하는 효과 때문에 global minima을 더욱 잘 찾을 수 있게 만든다고 주장함.

<img src="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/images/pmlc_0331.png?raw=true" width=480> <br/>
(source: [paper](https://arxiv.org/abs/1712.09913))

<img src="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/images/pmlc_0327.png?raw=true" width=300> <br/>
(A residual block: [practical-ml-vision-book](https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/images/pmlc_0327.png))



In [ ]:
# (ref) https://coding-yoon.tistory.com/141
x = torch.randn(1,256,224,224)
_, n_c, h, w =  x.size()

conv_path = nn.Sequential(
    nn.Conv2d(n_c, 64, kernel_size=1, stride=1, padding=0, bias=False),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False),
    nn.ReLU(),
    nn.Conv2d(64, n_c, kernel_size=1, stride=1, padding=0, bias=False),
)
relu = nn.ReLU()

out = conv_path(x) # C(x)
out = out + x  # C(x) + x ; residual mapping 
out = relu(out)

print(out.size())

torch.Size([1, 256, 224, 224])


## 2-2. macro architecture 
```macro architecture```란 ```block``` 모듈을 쌓는 컨셉이다 (i.e., block stacking).
* Inception, MobileNet, ShuffleNet 등에서 모듈 단위의 블록(block)을 쌓아서 네트워크 전체를 구성한다. 
* SqueezeNet도 이와 유사한 개념을 적용한다. 

<img src="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/images/pmlc_0324.png?raw=true" width=480> </br>
(source: [practical-ml-vision-book](https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/images/pmlc_0327.png))

# 3. Architecture
* Reference to [Hao Gao's medium](https://medium.com/@smallfishbigsea/notes-of-squeezenet-4137d51feef4) 

SqueezeNet을 구성하는 전략은 총 세가지다: 

1. $3\times3$ filter를 $1\times1$ filter로 대체한다; 
    * [#channel, H, W] → [#channel', H, W] 형태의 활성화 맵을 만들거면 $1 \times 1$ filter를 사용하는 것이 파라미터의 개수도 적기 때문에 저렴함 (전략 3과 관련). 
2. $3\times3$ filter의 채널 수를 줄인다.
    * $3\times3$ filter의 파라미터 개수 = $3\times3\times \text{num_channels}$ 이므로 채널이 늘어나면 파라미터 개수도 금증함. 
3. Convolution layer의 활성화 맵(activation map)을 크게 만들기 위해 downsample을 늦게 진행한다. 
    * 네트워크 앞쪽에서 downsample을 하게 되면 activation map의 크기가 줄어 연산량이 줄기 때문에 처리 속도는 빨라짐 
    * 하지만, 그만큼 activation map에서 정보가 소실되기 때문에 accuracy가 떨어짐 
    * 따라서, downsample을 늦게 하여 accuracy를 높이는 전략을 취함 

</br>

```전략 1,2```는 필터를 효율적으로 사용하여 최대한 <b>파라미터 개수를 줄이는 것을 목적</b>으로 한다. 이 과정에서 ```전략 3```은 <b>최대한 성능을 내기 위해 activation map의 크기를 유지하는 것</b>으로 전체 전략을 정리할 수 있다.  

## 3-1. Fire Module
* Reference to [Machine-Learning-Tokyo's github](https://github.com/Machine-Learning-Tokyo/DL-workshop-series/blob/master/Part%20I%20-%20Convolution%20Operations/ConvNets.ipynb)

SqueezeNet에서 block stacking을 위해 사용하는 모듈이다. <br>


* $1 \times 1$ conv filter를 통해 채널을 압축한다. <b>[Squeeze layer]</b>


* $1 \times 1$ 및 $3 \times 3$ conv filter를 통해 다시 채널을 팽창시킨다 <b>[Expand layer]</b>
* Inception 모듈과 같이 $1 \times 1$ 및 $3 \times 3$ conv filter 로 구성된 <b>Multiple path</b>를 줌으로써 다양한 특징을 학습할 수 있게 했다. 
* Activation function은 ReLU를 사용한다. 



<img src="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/images/pmlc_0325.png?raw=true" width="480" /> </br>
(source: [practical-ml-vision-book](https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/images/pmlc_0327.png))

In [ ]:
# (ref) https://github.com/pytorch/vision/blob/main/torchvision/models/squeezenet.py

class Fire(nn.Module):
    def __init__(self, inplanes: int, squeeze_planes: int, 
                 expand1x1_planes: int, expand3x3_planes: int) -> None:
        super(Fire, self).__init__()    

        # squeeze layer (채널 축소)
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        # expand layer (채널 팽창)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat(
            [self.expand1x1_activation(self.expand1x1(x)), 
             self.expand3x3_activation(self.expand3x3(x))], 
             axis=1)

In [ ]:
input = torch.randn(1, 96, 224, 224)

fire = Fire(96, 16, 64, 64) # 96 -> 16 축소 
                            # 16 -> 64 팽창 
                            # concate; 64+64=128
out = fire(input)
print(out.size())

torch.Size([1, 128, 224, 224])


## 3-2. Block stacking
<img src="https://i.stack.imgur.com/0pOi4.png" width=640> </br>
(source: [StackExchange](https://datascience.stackexchange.com/questions/33114/what-does-depth-mean-in-the-squeezenet-architectural-dimensions-table))

In [ ]:
class SqueezeNet(nn.Module):
    def __init__(self, version: str = "1_0", num_classes: int = 1000, dropout: float = 0.5) -> None:
        super(SqueezeNet, self).__init__()
        
        self.num_classes = num_classes

        if version == "1_0":
            self.features = nn.Sequential(
                # 첫 input을 처리할 때에만 예외적인 Convolution 연산과 MaxPool을 적용한 뒤
                # 이후에는 Fire Module을 계속 적용함.  
                nn.Conv2d(3, 96, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                
                # (입력채널, squeeze 채널, 1x1 expand 채널, 3x3 expand 채널)
                Fire(96, 16, 64, 64),
                Fire(128, 16, 64, 64),
                Fire(128, 32, 128, 128),
                
                # late downsampling(MaxPool)을 적용함
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 32, 128, 128),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                # late downsampling(MaxPool)을 적용함
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(512, 64, 256, 256),
            )
        elif version == "1_1":
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(64, 16, 64, 64),
                Fire(128, 16, 64, 64),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(128, 32, 128, 128),
                Fire(256, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                Fire(512, 64, 256, 256),
            )
        else:
            # FIXME: Is this needed? SqueezeNet should only be called from the
            # FIXME: squeezenet1_x() functions
            # FIXME: This checking is not done for the other models
            raise ValueError(f"Unsupported SqueezeNet version {version}: 1_0 or 1_1 expected")

        # Final convolution is initialized differently from the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout), final_conv, nn.ReLU(inplace=True), nn.AdaptiveAvgPool2d((1, 1))
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.classifier(x)
        return torch.flatten(x, 1)

In [ ]:
input = torch.randn(1, 3, 224, 224)
model = SqueezeNet(num_classes= 1000, dropout = 0.5)

out = model(input)

print(out.size())
print(f"# of learnable parameters: {count_parameters(model)/1e6}M")

torch.Size([1, 1000])
# of learnable parameters: 1.248424M
